# Задание

Вам необходимо построить модель, которая на основании данных, поступающих каждую минуту, определяют качество продукции, производимое на обжиговой машине.
Обжиговая машина представляет собой агрегат, состоящий из 5 одинаковых по размеру камер, в каждой камере установлено по 3 датчика температур. Кроме этого, для данной задачи Вы собрали данные о высоте слоя сырья и его влажности. Высота слоя и влажность измеряются при входе сырья в машину. Сырье проходит через обжиговую машину за час.
Данные с показателями работы обжиговой машины содержатся в файле X_data.csv:

Качество продукции измеряется в лаборатории по пробам, которые забираются каждый час, данные по известным анализам содержатся в файле Y_train.csv. В файле указано время забора пробы, проба забирается на выходе из обжиговой машины.
Вы договорились с заказчиком, что оценкой модели будет являться показатель MAE, для оценки модели необходимо сгенерировать предсказания за период, указанный в файле Y_submit.csv (5808 предиктов).


In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta

In [2]:
import lightgbm as lgb

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [3]:
df_x = pd.read_csv('X_data.csv', sep=';')
df_y = pd.read_csv('Y_train.csv', sep=';', header=None)
df_pred = pd.read_csv('Y_submit.csv', sep=';', header=None)

In [4]:
df_x.head()

,Unnamed: 0,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
0,2015-01-01 00:00:00,212,210,211,347,353,347,474,473,481,346,348,355,241,241,243,167.85,9.22
1,2015-01-01 00:01:00,212,211,211,346,352,346,475,473,481,349,348,355,241,241,243,162.51,9.22
2,2015-01-01 00:02:00,212,211,211,345,352,346,476,473,481,352,349,355,242,241,242,164.99,9.22
3,2015-01-01 00:03:00,213,211,211,344,351,346,477,473,481,355,349,355,242,241,242,167.34,9.22
4,2015-01-01 00:04:00,213,211,211,343,350,346,478,473,482,358,349,355,243,241,242,163.04,9.22


In [5]:
df_y.head()

,0,1
0,2015-01-04 00:05:00,392
1,2015-01-04 01:05:00,384
2,2015-01-04 02:05:00,393
3,2015-01-04 03:05:00,399
4,2015-01-04 04:05:00,400


In [6]:
df_pred.head()

,0,1
0,2018-05-04 00:05:00,420
1,2018-05-04 01:05:00,420
2,2018-05-04 02:05:00,420
3,2018-05-04 03:05:00,420
4,2018-05-04 04:05:00,420


In [7]:
# Конвертируем первый столбец из строкового типа в метку дата-время для последующей обработки

df_x = df_x.rename(index=str, columns={'Unnamed: 0':'T_time'})
df_x['T_time'] = df_x['T_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

# Вычтем один час из каждой метки времени для массивов показателей качества, так как значения показателей относятся к 
# образцам поданым в обжиговую печи за час до взятия образцов на анализ
df_y = df_y.rename(index=str, columns={0:'T_time'})
df_y['T_time'] = df_y['T_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') + timedelta(hours=-1))

df_pred = df_pred.rename(index=str, columns={0:'T_time'})
df_pred['T_time'] = df_pred['T_time'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') + timedelta(hours=-1))

In [8]:
df_y.head()

,T_time,1
0,2015-01-03 23:05:00,392
1,2015-01-04 00:05:00,384
2,2015-01-04 01:05:00,393
3,2015-01-04 02:05:00,399
4,2015-01-04 03:05:00,400


In [9]:
# Подготовим новый массив в котором будут включены только данные от датчиков имеющие соответствующие им результаты качества
# на выходе после обжига
train_df = pd.merge(df_x, df_y, on='T_time').rename(index=str, columns={1:'quality'})

In [10]:
train_df.head()

,T_time,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data,quality
0,2015-01-03 23:05:00,266,339,263,335,326,341,495,497,535,316,346,336,234,238,239,154.87,4.73,392
1,2015-01-04 00:05:00,277,326,273,322,335,352,505,501,670,326,379,337,231,236,242,153.77,7.90,384
2,2015-01-04 01:05:00,277,253,272,320,333,355,500,501,687,337,396,335,234,242,230,158.27,6.96,393
3,2015-01-04 02:05:00,262,218,260,326,336,330,505,499,443,347,399,332,243,251,240,153.36,7.29,399
4,2015-01-04 03:05:00,243,238,252,327,329,308,520,498,540,342,387,334,257,258,246,153.21,7.11,400


# Оценка модели

In [11]:
# Разделим массив на данные по фичам и соответствующием им таргетам для подачи на вход алгоритма

train_df_x = train_df.drop(['T_time','quality'], axis=1)
train_df_y = train_df['quality']

In [12]:
%%time

# Сделаем первичную оценку качества модели с использованием кросс-валидации 
cvs = cross_val_score(lgb.LGBMRegressor(), train_df_x, train_df_y, cv=5, n_jobs=-1, scoring='neg_mean_absolute_error')
print(cvs, '\n', 'MEAN = ',cvs.mean())

[-9.21964129 -9.47372931 -9.71813911 -9.03196054 -9.68603722] 
 MEAN =  -9.4259014926
Wall time: 2.92 s


In [13]:
# Теперь подготовим данные для которых надо будет предсказываеть целевое значение - качество выпускаемой продукции

test_df = df_x[df_x['T_time'].isin(df_pred['T_time'])]

In [14]:
test_df.head()

,T_time,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data,AH_data
1755305,2018-05-03 23:05:00,254,257,256,349,350,355,451,454,439,347,342,331,255,266,245,157.46,6.39
1755365,2018-05-04 00:05:00,258,261,256,347,337,352,474,434,443,336,321,516,248,247,236,154.93,6.81
1755425,2018-05-04 01:05:00,255,263,258,351,-162,359,460,441,454,320,319,341,238,255,236,156.52,7.89
1755485,2018-05-04 02:05:00,259,261,288,363,104,374,482,474,483,343,325,307,226,249,255,153.37,6.16
1755545,2018-05-04 03:05:00,272,266,279,373,338,386,495,510,503,330,307,323,213,245,250,161.45,7.53


In [15]:
# Перебор гиперпараметров можно выполнять через различные инструменты scikit, процесс это достаточно времязатратный, 
# вынесем его за скобки данного задания, ограничившись значениями по умолчанию.

param = {
    #'bagging_freq': 5,
    #'bagging_fraction': 0.33,
    #'boost_from_average':'false',
    #'boost': 'gbdt',
    #'feature_fraction': 0.04,
    #'learning_rate': 0.001,
    #'max_depth': -1,
    'metric':'mae',
    #'min_data_in_leaf': 80,
    #'min_sum_hessian_in_leaf': 10.0,
    #'num_leaves': 13,
    'num_threads': 8,
    #'tree_learner': 'serial',
    'objective': 'regression',
    #'regression_l1': 'mae',
    'verbosity': 1
}

In [16]:
features = [c for c in train_df.columns if c not in ['T_time', 'quality']]
target = train_df['quality']

In [17]:
# Выполним стратифицированние разбиение множества на 5 частей для кросс-валидации

num_round = 999999
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=101)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))

In [18]:
%%time

# Обучим нашу модель, рассчитаем среднее значение абсолютной ошибки усредненное по всем фолдам, а так же 
# рассчитаем предсказанное значение качества изделий

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 5000)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("\nCV score: {:<8.5f}".format(mean_absolute_error(target, oof)))

C:\Users\User\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Fold 0
Training until validation scores don't improve for 5000 rounds.
[1000]	training's l1: 4.06739	valid_1's l1: 8.88727
[2000]	training's l1: 2.20643	valid_1's l1: 8.91565
[3000]	training's l1: 1.25755	valid_1's l1: 8.93388
[4000]	training's l1: 0.721728	valid_1's l1: 8.94409
[5000]	training's l1: 0.423985	valid_1's l1: 8.95296
Early stopping, best iteration is:
[943]	training's l1: 4.21869	valid_1's l1: 8.88034
Fold 1
Training until validation scores don't improve for 5000 rounds.
[1000]	training's l1: 4.06422	valid_1's l1: 8.77779
[2000]	training's l1: 2.19783	valid_1's l1: 8.82552
[3000]	training's l1: 1.2404	valid_1's l1: 8.83844
[4000]	training's l1: 0.719746	valid_1's l1: 8.85045
[5000]	training's l1: 0.42525	valid_1's l1: 8.85276
Early stopping, best iteration is:
[862]	training's l1: 4.44987	valid_1's l1: 8.75615
Fold 2
Training until validation scores don't improve for 5000 rounds.
[1000]	training's l1: 4.03322	valid_1's l1: 8.93204
[2000]	training's l1: 2.19934	valid_1's l

In [19]:
# Сохраним результат предсказаний нашей модели в файл

submission = pd.DataFrame({"T_time": test_df.T_time.values})
submission["quality"] = predictions
submission.to_csv("submission.csv", index=False)

In [20]:
submission.head(10)

,T_time,quality
0,2018-05-03 23:05:00,451.124655
1,2018-05-04 00:05:00,440.761956
2,2018-05-04 01:05:00,443.797764
3,2018-05-04 02:05:00,425.431875
4,2018-05-04 03:05:00,403.057090
5,2018-05-04 04:05:00,409.881373
6,2018-05-04 05:05:00,408.740384
7,2018-05-04 06:05:00,393.981277
8,2018-05-04 07:05:00,360.262549
9,2018-05-04 08:05:00,378.448788
